# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1bb8702f10>
├── 'a' --> tensor([[-0.7650, -0.2770,  0.4788],
│                   [ 1.0649,  0.8764, -0.7595]])
└── 'x' --> <FastTreeValue 0x7f1bb8702df0>
    └── 'c' --> tensor([[ 0.9560,  0.5494,  0.2953, -0.9362],
                        [ 0.1601,  0.5042, -0.3439, -0.7823],
                        [-1.6321,  1.9108, -0.2414, -0.1341]])

In [4]:
t.a

tensor([[-0.7650, -0.2770,  0.4788],
        [ 1.0649,  0.8764, -0.7595]])

In [5]:
%timeit t.a

61.3 ns ± 0.0415 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1bb8702f10>
├── 'a' --> tensor([[-1.0415, -0.8927,  0.3039],
│                   [-0.2076, -0.0214,  0.9917]])
└── 'x' --> <FastTreeValue 0x7f1bb8702df0>
    └── 'c' --> tensor([[ 0.9560,  0.5494,  0.2953, -0.9362],
                        [ 0.1601,  0.5042, -0.3439, -0.7823],
                        [-1.6321,  1.9108, -0.2414, -0.1341]])

In [7]:
%timeit t.a = new_value

67.9 ns ± 0.0323 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.7650, -0.2770,  0.4788],
               [ 1.0649,  0.8764, -0.7595]]),
    x: Batch(
           c: tensor([[ 0.9560,  0.5494,  0.2953, -0.9362],
                      [ 0.1601,  0.5042, -0.3439, -0.7823],
                      [-1.6321,  1.9108, -0.2414, -0.1341]]),
       ),
)

In [10]:
b.a

tensor([[-0.7650, -0.2770,  0.4788],
        [ 1.0649,  0.8764, -0.7595]])

In [11]:
%timeit b.a

58.6 ns ± 0.0531 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.3724,  0.7445, -0.3935],
               [-1.7135,  1.1183, -0.7893]]),
    x: Batch(
           c: tensor([[ 0.9560,  0.5494,  0.2953, -0.9362],
                      [ 0.1601,  0.5042, -0.3439, -0.7823],
                      [-1.6321,  1.9108, -0.2414, -0.1341]]),
       ),
)

In [13]:
%timeit b.a = new_value

479 ns ± 0.125 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

874 ns ± 1.65 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 36.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 286 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 358 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f1bb8702ee0>
├── 'a' --> tensor([[[-0.7650, -0.2770,  0.4788],
│                    [ 1.0649,  0.8764, -0.7595]],
│           
│                   [[-0.7650, -0.2770,  0.4788],
│                    [ 1.0649,  0.8764, -0.7595]],
│           
│                   [[-0.7650, -0.2770,  0.4788],
│                    [ 1.0649,  0.8764, -0.7595]],
│           
│                   [[-0.7650, -0.2770,  0.4788],
│                    [ 1.0649,  0.8764, -0.7595]],
│           
│                   [[-0.7650, -0.2770,  0.4788],
│                    [ 1.0649,  0.8764, -0.7595]],
│           
│                   [[-0.7650, -0.2770,  0.4788],
│                    [ 1.0649,  0.8764, -0.7595]],
│           
│                   [[-0.7650, -0.2770,  0.4788],
│                    [ 1.0649,  0.8764, -0.7595]],
│           
│                   [[-0.7650, -0.2770,  0.4788],
│                    [ 1.0649,  0.8764, -0.7595]]])
└── 'x' --> <FastTreeValue 0x7f1b0e9a0d00>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 67.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1b0e3680d0>
├── 'a' --> tensor([[-0.7650, -0.2770,  0.4788],
│                   [ 1.0649,  0.8764, -0.7595],
│                   [-0.7650, -0.2770,  0.4788],
│                   [ 1.0649,  0.8764, -0.7595],
│                   [-0.7650, -0.2770,  0.4788],
│                   [ 1.0649,  0.8764, -0.7595],
│                   [-0.7650, -0.2770,  0.4788],
│                   [ 1.0649,  0.8764, -0.7595],
│                   [-0.7650, -0.2770,  0.4788],
│                   [ 1.0649,  0.8764, -0.7595],
│                   [-0.7650, -0.2770,  0.4788],
│                   [ 1.0649,  0.8764, -0.7595],
│                   [-0.7650, -0.2770,  0.4788],
│                   [ 1.0649,  0.8764, -0.7595],
│                   [-0.7650, -0.2770,  0.4788],
│                   [ 1.0649,  0.8764, -0.7595]])
└── 'x' --> <FastTreeValue 0x7f1bb869f4f0>
    └── 'c' --> tensor([[ 0.9560,  0.5494,  0.2953, -0.9362],
                        [ 0.1601,  0.5042, -0.3439, -0.7823],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 56.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60.3 µs ± 53.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.7650, -0.2770,  0.4788],
                [ 1.0649,  0.8764, -0.7595]],
       
               [[-0.7650, -0.2770,  0.4788],
                [ 1.0649,  0.8764, -0.7595]],
       
               [[-0.7650, -0.2770,  0.4788],
                [ 1.0649,  0.8764, -0.7595]],
       
               [[-0.7650, -0.2770,  0.4788],
                [ 1.0649,  0.8764, -0.7595]],
       
               [[-0.7650, -0.2770,  0.4788],
                [ 1.0649,  0.8764, -0.7595]],
       
               [[-0.7650, -0.2770,  0.4788],
                [ 1.0649,  0.8764, -0.7595]],
       
               [[-0.7650, -0.2770,  0.4788],
                [ 1.0649,  0.8764, -0.7595]],
       
               [[-0.7650, -0.2770,  0.4788],
                [ 1.0649,  0.8764, -0.7595]]]),
    x: Batch(
           c: tensor([[[ 0.9560,  0.5494,  0.2953, -0.9362],
                       [ 0.1601,  0.5042, -0.3439, -0.7823],
                       [-1.6321,  1.9108, -0.2414, -0.1341]],
         

In [26]:
%timeit Batch.stack(batches)

78.4 µs ± 75.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.7650, -0.2770,  0.4788],
               [ 1.0649,  0.8764, -0.7595],
               [-0.7650, -0.2770,  0.4788],
               [ 1.0649,  0.8764, -0.7595],
               [-0.7650, -0.2770,  0.4788],
               [ 1.0649,  0.8764, -0.7595],
               [-0.7650, -0.2770,  0.4788],
               [ 1.0649,  0.8764, -0.7595],
               [-0.7650, -0.2770,  0.4788],
               [ 1.0649,  0.8764, -0.7595],
               [-0.7650, -0.2770,  0.4788],
               [ 1.0649,  0.8764, -0.7595],
               [-0.7650, -0.2770,  0.4788],
               [ 1.0649,  0.8764, -0.7595],
               [-0.7650, -0.2770,  0.4788],
               [ 1.0649,  0.8764, -0.7595]]),
    x: Batch(
           c: tensor([[ 0.9560,  0.5494,  0.2953, -0.9362],
                      [ 0.1601,  0.5042, -0.3439, -0.7823],
                      [-1.6321,  1.9108, -0.2414, -0.1341],
                      [ 0.9560,  0.5494,  0.2953, -0.9362],
                      [ 0.1601,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 356 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

308 µs ± 2.33 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
